In [1]:
import pandas as pd
from tensorflow import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
dfe = pd.read_csv("../Data/Data_cleaned/emotion_cleaned_rudy.csv")[["target", "clean_text"]].dropna()

FileNotFoundError: [Errno 2] No such file or directory: '../Data/Data_cleaned/emotion_cleaned_rudy.csv'

In [4]:
encoder = LabelEncoder()
dfe["cible"] = encoder.fit_transform(dfe["target"])

In [5]:
liste = dfe["target"].unique()

## Spliting des données

In [2]:
X_train, X_test, y_train, y_test = train_test_split(dfe["clean_text"], dfe["cible"], train_size=0.8, random_state=1, stratify=dfe["target"])
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.5, stratify=y_train)

NameError: name 'train_test_split' is not defined

## Vectorisation

In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

In [8]:
vectorizer =CountVectorizer()
vectorizer.fit(X_train)

CountVectorizer()

In [1]:
X_train_t = vectorizer.transform(X_train).toarray()
X_test_t = vectorizer.transform(X_test).toarray()
X_val_t = vectorizer.transform(X_val).toarray()

NameError: name 'vectorizer' is not defined

## Neural Network

In [63]:
model = keras.Sequential()

initializer = keras.initializers.HeNormal()
regularizer = keras.regularizers.L2(0.005)

model.add(keras.layers.Dense(128, input_dim=X_train_t.shape[1], activation="relu",
                             kernel_initializer=initializer,kernel_regularizer=regularizer))
model.add(keras.layers.Dense(64,kernel_regularizer=regularizer, activation="relu"))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(6, activation="softmax"))

In [64]:
model.compile(
    loss= keras.losses.SparseCategoricalCrossentropy(),
    optimizer= keras.optimizers.Adam(learning_rate=0.0001),
    metrics=["accuracy"]
)

In [65]:
history = model.fit(
    X_train_t,
    y_train,
    epochs=100,
    callbacks=keras.callbacks.EarlyStopping(patience=3, monitor="val_loss"),
    batch_size=16,
    validation_data=(X_val_t, y_val)
)

Epoch 1/100
537/537 [==============================] - 10s 17ms/step - loss: 2.2593 - accuracy: 0.3236 - val_loss: 1.8513 - val_accuracy: 0.3504
Epoch 2/100
537/537 [==============================] - 9s 17ms/step - loss: 1.7230 - accuracy: 0.4346 - val_loss: 1.6171 - val_accuracy: 0.5450
Epoch 3/100
537/537 [==============================] - 9s 16ms/step - loss: 1.4743 - accuracy: 0.5851 - val_loss: 1.3873 - val_accuracy: 0.6261
Epoch 4/100
537/537 [==============================] - 9s 17ms/step - loss: 1.2267 - accuracy: 0.7235 - val_loss: 1.2141 - val_accuracy: 0.7461
Epoch 5/100
537/537 [==============================] - 9s 16ms/step - loss: 1.0712 - accuracy: 0.7990 - val_loss: 1.1120 - val_accuracy: 0.7845
Epoch 6/100
537/537 [==============================] - 9s 17ms/step - loss: 0.9749 - accuracy: 0.8370 - val_loss: 1.0490 - val_accuracy: 0.8065
Epoch 7/100
537/537 [==============================] - 9s 17ms/step - loss: 0.9104 - accuracy: 0.8637 - val_loss: 1.0012 - val_accuracy

KeyboardInterrupt: 

In [14]:
from sklearn.metrics import f1_score
import numpy as np

In [15]:
print(45*'-'+"weighted"+46*'-')
print("Train f1_score:", f1_score(np.argmax(model.predict(X_train_t), axis=1), y_train, average="weighted"))
print("Val f1_score:", f1_score(np.argmax(model.predict(X_val_t), axis=1), y_val, average="weighted"))
print(47*'-'+"macro"+47*'-')
print("Train f1_score:", f1_score(np.argmax(model.predict(X_train_t), axis=1), y_train, average="macro"))
print("Val f1_score:", f1_score(np.argmax(model.predict(X_val_t), axis=1), y_val, average="macro"))
print(47*'-'+"micro"+47*'-')
print("Train f1_score:", f1_score(np.argmax(model.predict(X_train_t), axis=1), y_train, average="micro"))
print("Val f1_score:", f1_score(np.argmax(model.predict(X_val_t), axis=1), y_val, average="micro"))

---------------------------------------------weighted----------------------------------------------
Train f1_score: 0.9955798050252695
Val f1_score: 0.8770163151831581
-----------------------------------------------macro-----------------------------------------------
Train f1_score: 0.9925254548986157
Val f1_score: 0.836709445845622
-----------------------------------------------micro-----------------------------------------------
Train f1_score: 0.9955721277091587
Val f1_score: 0.8756700069913773


## F1 score weighted par classe

In [16]:
data_score = pd.DataFrame()
data_score["y_pred"] = np.argmax(model.predict(X_val_t),axis=1)
data_score["y_true"] = y_val.reset_index()["cible"]

In [17]:
def f1_score_classe(df,classe):
    df_score = df[df.y_true==classe]
    return f1_score(df_score["y_pred"],df_score["y_true"], average="weighted")

In [18]:
for i in range(6):
    print(f"Classe {liste[i]} : {f1_score_classe(data_score,i)}")

Classe sadness : 0.7758226637483603
Classe anger : 0.7388747802972582
Classe love : 0.9003156920208354
Classe surprise : 0.6346580249019274
Classe fear : 0.8720711169106552
Classe happy : 0.5296901221558755


## TextVectorization


In [19]:
max_features = 20000
embedding_dim = 256
sequence_length = 100

In [74]:
from tensorflow.keras.layers import TextVectorization

In [75]:
vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

In [76]:
vectorize_layer.adapt(X_train)

In [77]:
train_ds = vectorizer.transform(X_train)
test_ds = vectorizer.transform(X_test)
val_ds = vectorizer.transform(X_val)

In [109]:
train_ds.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [104]:
inputs = tf.keras.Input(shape=(None,), dtype="int64", name='text')

x = keras.layers.Embedding(max_features, embedding_dim)(inputs)
# x = keras.layers.Dense(128, activation="relu")(x)
x = keras.layers.Conv1D(128, 7, padding="valid", activation="relu")(x)
# x = keras.layers.Conv1D(128, 7, padding="same", activation="relu", strides=3)(x)

# x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(64, activation="relu")(x)
# x = keras.layers.BatchNormalization()(x)
outputs = keras.layers.Dense(6, activation="softmax")(x)

model = tf.keras.Model(inputs, outputs)

In [114]:
y_train=np.array(y_train)
y_test=np.array(y_test)
y_val=np.array(y_val)

In [118]:
y_train.shape, train_ds.toarray().shape

((8582,), (8582, 11546))

In [115]:
model.compile(
    loss= keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"]
)

In [116]:
model.fit(
    train_ds.toarray(),
    y_train,
    epochs=50,
    batch_size=8,
    callbacks=[keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)],
    validation_data=(val_ds.toarray(),y_val)
)

Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "c:\Users\Apprenant\anaconda3\envs\rb_ds\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Apprenant\anaconda3\envs\rb_ds\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\Apprenant\anaconda3\envs\rb_ds\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "c:\Users\Apprenant\anaconda3\envs\rb_ds\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "c:\Users\Apprenant\anaconda3\envs\rb_ds\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "c:\Users\Apprenant\anaconda3\envs\rb_ds\lib\site-packages\tornado\platform\asyncio.py", 